In [2]:
import pandas as pd


processed_dir = "../../Data/Tezos/Processed/"
raw_dir = "../../Data/Tezos/Raw_data/"
final_dir = "../../Data/Tezos/Final/"

In [4]:
df_blocks = pd.read_csv(raw_dir + "TZ_Blocks.csv")

df_level_time = df_blocks[["level", "timestamp"]]
df_level_time.timestamp = pd.to_datetime(df_level_time.timestamp, format="%Y-%m-%dT%H:%M:%SZ")
df_level_time["date"] = df_level_time.timestamp.apply(lambda x: x.date())

df_level_time.drop(["timestamp"], axis=1, inplace=True)


/tmp/ipykernel_34596/3674169998.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_time.timestamp = pd.to_datetime(df_level_time.timestamp, format="%Y-%m-%dT%H:%M:%SZ")
/tmp/ipykernel_34596/3674169998.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_time["date"] = df_level_time.timestamp.apply(lambda x: x.date())
/tmp/ipykernel_34596/3674169998.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

## Convert contract per level data to daily

In [ ]:
df_contracts = pd.read_csv(processed_dir + "Contracts_per_level.csv")


In [ ]:
df_result = pd.merge(df_contracts, df_level_time, how="left", on="level")

/tmp/ipykernel_1288264/2576669266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_time.drop(["timestamp"], axis=1, inplace=True)


In [18]:
df_result = df_result[~df_result.date.isna()]

In [22]:
agg = {}
for i in ['new_assets', 'new_delegator_contracts', 'new_smart_contracts']:
    agg[i] = "sum"

for i in ['total_assets', 'total_delegator_contracts', 'total_smart_contracts']:
    agg[i] = "last"

daily_stats = df_result.groupby("date").agg(agg).reset_index()

In [26]:
daily_stats.to_csv(final_dir + "Contract_data_daily.csv", index= False)

## Convert voting data from block to daily stats

In [5]:
df_voting = pd.read_csv(processed_dir + "Voting_by_block.csv")

In [35]:
df_result2 = pd.merge(df_voting, df_level_time, how="left", on="level")

In [39]:
df_result2 = df_result2[~df_result2.date.isna()]
df_result2.fillna(0, inplace=True)

agg = {}
for i in df_result2.columns:
    if i=="level" or i=="date": continue
    agg[i] = "last"

daily_stats_voting = df_result2.groupby("date").agg(agg).reset_index()

In [42]:
daily_stats_voting.to_csv(final_dir + "Voting_daily_stats.csv", index = False)